# IMDB 영화 리뷰 감성 분석 (자연어 처리)

## 1. 라이브러리 임포트

In [ ]:
import requests
import subprocess
import re
import string
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import os, pathlib, shutil, random
import keras

## 2. 데이터 다운로드 및 압축 해제

Stanford 대학의 AI 연구실에서 제공하는 IMDB 영화 리뷰 데이터셋을 다운로드하고 압축을 해제합니다.

In [ ]:
# 데이터 다운로드 함수
def download():
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"

    response = requests.get(url, stream=True)  # 스트리밍 방식으로 다운로드
    with open(file_name, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):  # 8KB씩 다운로드
            file.write(chunk)

    print("Download complete!")

# 압축풀기 함수 (tar 프로그램 필요)
def release():
    # tar.gz => linux에서는 파일을 여러개를 한번에 압축을 못함 tar라는 형식으로 압축할 모든 파일을 하나로 묶어서 패키지로 만든다음에 
    #           압축을 한다.  tar , gz가동  그래서 압축풀고 다시 패키지도 풀어야 한다. 
    #           tar  -xvzf 파일명   형태임
    subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"], shell=True) #tar 프로그램 가동하기 
    print("압축풀기 완료")

In [ ]:
# download() # 최초 한번만 실행
# release()  # 최초 한번만 실행

## 3. 데이터셋 준비: 훈련/검증 데이터 분리

원본 훈련 데이터셋(train)의 20%를 검증 데이터셋(validation)으로 분리합니다. `unsup` 폴더는 라벨이 없으므로 사용하지 않습니다 (미리 수동으로 삭제했다고 가정).

In [ ]:
# 라벨링 및 데이터 분리 함수
def labeling(): 
    base_dir = pathlib.Path("aclImdb") 
    val_dir = base_dir/"val"   # pathlib 객체에  / "디렉토리" => 결과가 문자열이 아니다 
    train_dir = base_dir/"train"

    # val 디렉토리 생성
    for category in ("neg", "pos"):
        if not os.path.exists(val_dir/category):
             os.makedirs(val_dir/category)  #디렉토리를 만들고 
        
        files = os.listdir(train_dir/category) #해당 카테고리의 파일 목록을 모두 가져온다 
        random.Random(1337).shuffle(files) #파일을 랜덤하게 섞어서 복사하려고 파일 목록을 모두 섞는다 
        num_val_samples = int(0.2 * len(files)) 
        val_files = files[-num_val_samples:] #20%만 val폴더로 이동한다 
        for fname in val_files:
            shutil.move(train_dir/category/fname, val_dir/category/fname )

In [ ]:
# labeling() # 최초 한번만 실행

## 4. 데이터셋 로드

`keras.utils.text_dataset_from_directory`를 사용하여 디렉토리에서 텍스트 파일을 `tf.data.Dataset` 객체로 로드합니다.

In [ ]:
batch_size = 32 #한번에 읽어올 양 
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", #디렉토리명 
    batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", #디렉토리명 
    batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", #디렉토리명 
    batch_size=batch_size
)

### 데이터셋 구조 확인

로드된 데이터셋의 형태(shape)와 데이터 타입(dtype)을 확인합니다. 데이터셋은 (텍스트, 라벨) 쌍으로 구성됩니다. 라벨은 폴더 이름(neg, pos)에 따라 자동으로 0과 1로 인코딩됩니다.

In [ ]:
for inputs, targets in train_ds: #실제 읽어오는 데이터 확인 
    print("inputs.shape", inputs.shape)
    print("inputs.dtype", inputs.dtype)
    print("targets.shape", targets.shape)
    print("targets.dtype", targets.dtype)
    print("inputs[0]", inputs[:3])
    print("targets[0]", targets[:3])
    break #하나만 출력해보자

## 5. 텍스트 벡터화 (Text Vectorization)

텍스트 데이터를 모델이 처리할 수 있는 정수 시퀀스로 변환합니다.

- `max_tokens`: 어휘 사전에 포함할 최대 단어 수 (빈도수 기준)
- `output_sequence_length`: 모든 시퀀스를 동일한 길이로 맞추기 위한 최대 길이

In [ ]:
max_length = 600  #한 평론에서 사용하는 단어는 최대 길이를 600개라고 보자  
max_tokens = 20000 #자주 사용하는 단어 20000 개만 쓰겠다 

text_vectorization = TextVectorization( 
    max_tokens = max_tokens,
    output_mode = "int", #임베딩 층을 사용하려면 반드시 int여야 한다
    output_sequence_length = max_length  
)

### 어휘 사전 생성

훈련 데이터셋의 텍스트만 사용하여 `TextVectorization` 레이어의 어휘 사전을 생성합니다 (`adapt` 메서드 사용).

In [ ]:
# 텍스트 데이터만 있는 데이터셋을 만듭니다.
text_only_train_ds = train_ds.map(lambda x, y: x)
# 어휘 사전을 만듭니다.
text_vectorization.adapt(text_only_train_ds)

### 데이터셋에 벡터화 적용

생성된 어휘 사전을 사용하여 훈련, 검증, 테스트 데이터셋의 텍스트를 모두 정수 시퀀스로 변환합니다.

In [ ]:
int_train_ds = train_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=4 )
int_val_ds = val_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=4 )
int_test_ds = test_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=4 )

### 변환된 데이터 확인

In [ ]:
for item in int_train_ds:
    print(item)
    break 

## 6. 모델 구축 (RNN)

이전 예제에서는 정수 시퀀스를 직접 원-핫 인코딩하여 모델에 입력했지만, 이는 메모리 비효율을 야기할 수 있습니다. 이번에는 **임베딩 레이어**를 사용하는 대신, 원-핫 인코딩을 모델의 일부로 포함시키는 `Lambda` 레이어를 사용해봅니다. (실제로는 임베딩 레이어를 사용하는 것이 일반적입니다.)

- `Input`: 정수 시퀀스를 입력으로 받습니다.
- `Lambda`: 입력된 정수 시퀀스를 `tf.one_hot`을 사용하여 원-핫 벡터 시퀀스로 변환합니다.
- `Bidirectional(LSTM)`: 양방향 LSTM을 사용하여 시퀀스의 문맥을 양방향으로 학습합니다.
- `Dropout`: 과적합을 방지하기 위해 일부 뉴런을 비활성화합니다.
- `Dense`: 최종 출력을 생성하여 긍정(1) 또는 부정(0)을 예측합니다.

In [ ]:
from keras import models, layers

inputs = keras.Input(shape=(None,), dtype="int64")

# 원-핫 인코딩을 모델의 한 레이어로 추가
# 시퀀스 => 원-핫 인코딩으로 변환하는 Lambda 레이어
embedded = layers.Lambda(
    lambda x: tf.reshape(tf.one_hot(x, depth=max_tokens), (-1, tf.shape(x)[1], max_tokens)),  
    output_shape=(None, max_tokens) 
)(inputs) 

# 양방향 RNN 모델 구성
x = layers.Bidirectional( layers.LSTM(32))(embedded) 
x = layers.Dropout(0.5)(x) 
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs) 

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## 7. 모델 훈련

In [ ]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)

## 8. 모델 평가

In [ ]:
print("테스트셋 평가 결과 ", model.evaluate(int_test_ds))